In [2]:
%load_ext autoreload
%autoreload 2

# Then your regular imports
import logging
import pandas as pd
from utils import prepare_df, group_cases_by_trace
from incremental_softmax_recovery import incremental_softmax_recovery

In [3]:
# Configure root logger to show DEBUG messages
logging.basicConfig(
    level=logging.DEBUG,          # Set to DEBUG (lowest level)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Optional: Custom format for timestamps/names
    force=True                    # Force override of any existing handlers (useful in Jupyter)
)


In [4]:
# load your DataFrame and softmax list
result = prepare_df('50salads')
if len(result) == 2:
    df, softmax_lst = result
else:
    df, softmax_lst, _ = result

# group by trace and inspect
trace_groups = group_cases_by_trace(df)
trace_groups

,case_list,trace_length
0,"[0, 1, 2, 3]",5687
1,"[32, 33, 34, 35]",6186
2,"[36, 37, 38, 39]",5840
3,"[28, 29, 30, 31]",5261
4,"[4, 5, 6, 7]",6208
5,"[16, 17, 18, 19]",6293
6,"[24, 25, 26, 27]",6046
7,"[8, 9, 10, 11]",6584
8,"[12, 13, 14, 15]",5558
9,"[20, 21, 22, 23]",5792


In [12]:
# Correct configuration for incremental_softmax_recovery
config = {
    # === Data Splitting ===
    'n_train_traces': 10,                     # Number of training traces
    'n_test_traces': 10,                     # Number of test traces  
    'train_cases': None,                     # Specific train case IDs (overrides n_train_traces)
    'test_cases': None,                      # Specific test case IDs (overrides n_test_cases)
    'ensure_train_variant_diversity': True,  # Enforce distinct variants in training
    'ensure_test_variant_diversity': False,  # Enforce distinct variants in testing
    
    # === Sampling Configuration ===
    'sequential_sampling': True,             # True: sample from activity runs, False: uniform sampling
    'n_indices': None,                       # Events to sample per trace (when sequential_sampling=False)
    'n_per_run': 4,                          # Events per activity run (when sequential_sampling=True)
    'independent_sampling': True,            # Each trace gets different random seed
    
    # === Beam Search ===
    'beam_width': 1,                        # Number of candidates to maintain
    'activity_prob_threshold': 0.0,          # Minimum probability to consider activity
    'beam_score_alpha': 1.0,                # Weight between avg cost and total cost
    'completion_patience': 20,                # Extra iterations after first completion
    
    # === Cost Function ===
    'cost_function': "linear",               # "linear", "logarithmic", or callable
    'model_move_cost': 1.0,                  # Cost for model-only moves
    'log_move_cost': 1.0,                    # Cost for log-only moves  
    'tau_move_cost': 0.0,                   # Cost for silent (tau) moves
    'non_sync_penalty': 1.0,                 # Penalty for non-sync moves
    
    # === Conditional Probabilities ===
    'use_cond_probs': True,                  # Enable conditional probabilities
    'max_hist_len': 3,                       # Maximum history length for conditioning
    'lambdas': [0.1, 0.3, 0.6],              # Blending weights for n-gram smoothing
    'alpha': 0.5,                            # History vs base probability weight (0=history, 1=base)
    'use_ngram_smoothing': True,             # Apply n-gram smoothing
    
    # === Temperature Calibration ===
    'use_calibration': True,                 # Enable temperature scaling
    'temp_bounds': (1.0, 10.0),              # Temperature optimization bounds
    'temperature': None,                     # Manual temperature (bypasses optimization)
    
    # === Logging ===
    'verbose': True,                          # Enable logging output
    'log_level': logging.INFO,                # Logging level (logging.DEBUG for more details)
    
    # === Miscellaneous ===
    'round_precision': 2,                     # Decimal places for probability rounding
    'random_seed': 321,                       # Random seed for reproducibility
    'save_model_path': "./discovered_petri_net",  # Path for saved model (without extension)
    'save_model': True,                     # Save model to PDF (set to True if you want visualization)
}

# Usage:
output = incremental_softmax_recovery(
    df=df,
    softmax_lst=softmax_lst,
    **config 
)

# Unpack results
results_df, accuracy_dict, prob_dict = output

2025-07-22 11:13:32,308 - incremental_softmax_recovery - INFO - Starting incremental softmax recovery.
2025-07-22 11:13:32,324 - incremental_softmax_recovery - INFO - Validated sequential case IDs (found 40 unique cases) and 40 softmax matrices.
2025-07-22 11:13:32,326 - incremental_softmax_recovery - INFO - Validated sampling parameters: sequential runs with n_per_run=4.
2025-07-22 11:13:32,326 - incremental_softmax_recovery - INFO - Validated input parameters: beam_width=1, alpha=0.5, round_precision=2.
2025-07-22 11:13:32,327 - incremental_softmax_recovery - INFO - Prepared cost function: linear (model=1.0, log=1.0, tau=0.0).
2025-07-22 11:13:32,327 - incremental_softmax_recovery - INFO - Prepared softmax arrays: 40 traces with individual shape (19, 5687).
2025-07-22 11:13:32,749 - incremental_softmax_recovery - INFO - Filtered log and softmax matrices: 237820 -> 3344 events (1.4% retained).
2025-07-22 11:13:32,774 - incremental_softmax_recovery - INFO - Performed train/test split: 

PNG visualization saved to: discovered_petri_net.png
PDF visualization saved to: discovered_petri_net.pdf


2025-07-22 11:14:56,021 - classes - INFO - Built marking transition map with 1450 markings
2025-07-22 11:14:56,021 - incremental_softmax_recovery - INFO - Computed marking-to-transition map with 1450 reachable markings.
2025-07-22 11:14:56,025 - incremental_softmax_recovery - INFO - Built conditional probability dictionary: 274 histories, avg 1.8 activities per history.
2025-07-22 11:14:56,025 - incremental_softmax_recovery - INFO - Built conditional probability dictionary.
2025-07-22 11:14:56,036 - incremental_softmax_recovery - INFO - Prepared 10 test softmax matrices with calibration.
2025-07-22 11:14:56,036 - incremental_softmax_recovery - INFO - Extracted 10 test case IDs for processing.
2025-07-22 11:14:56,038 - incremental_softmax_recovery - DEBUG - Case 1/10: 28
2025-07-22 11:14:56,079 - beam_search - INFO - Active beam is empty. Stopping search.
2025-07-22 11:14:56,079 - beam_search - INFO - Beam search finished. Found 1 candidates that completed the trace.
2025-07-22 11:14:56

In [9]:
# Compute average accuracy for each metric
avg_sktr_accuracy = sum(accuracy_dict['sktr_accuracy']) / len(accuracy_dict['sktr_accuracy'])
avg_argmax_accuracy = sum(accuracy_dict['argmax_accuracy']) / len(accuracy_dict['argmax_accuracy'])

print(f"Average SKTR Accuracy: {avg_sktr_accuracy:.4f}")
print(f"Average Argmax Accuracy: {avg_argmax_accuracy:.4f}")

# Show original dictionary for reference
accuracy_dict

Average SKTR Accuracy: 0.0654
Average Argmax Accuracy: 0.8181


{'sktr_accuracy': [0.06944444444444445,
  0.08333333333333333,
  0.08333333333333333,
  0.05434782608695652,
  0.06944444444444445,
  0.05,
  0.06578947368421052,
  0.056818181818181816,
  0.052083333333333336,
  0.06944444444444445],
 'argmax_accuracy': [0.875,
  0.7666666666666667,
  0.7833333333333333,
  0.8043478260869565,
  0.9166666666666666,
  0.67,
  0.7236842105263158,
  0.9090909090909091,
  0.7604166666666666,
  0.9722222222222222]}

In [33]:
prob_dict[('17',)]

{'17': 0.75, '8': 0.03, '2': 0.03, '0': 0.05, '7': 0.07, '11': 0.05, '6': 0.03}

In [13]:
# Show all rows for case '28' without truncation, including full list values
case_28_df = results_df[results_df['case:concept:name'] == '28']
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None, 'display.width', None):
    display(case_28_df)
total_cost = case_28_df['sktr_move_cost'].sum()
print(f"Total SKTR move cost for case 28: {total_cost:.4f}")

,case:concept:name,step,sktr_activity,argmax_activity,ground_truth,all_probs,all_activities,is_correct,cumulative_accuracy,sktr_move_cost
0,28,0,17,17,17,"[0.05, 0.05, 0.05, 0.04, 0.05, 0.06, 0.05, 0.04, 0.05, 0.05, 0.05, 0.05, 0.04, 0.05, 0.04, 0.05, 0.04, 0.1, 0.07]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.45
1,28,1,17,17,17,"[0.05, 0.05, 0.05, 0.04, 0.05, 0.06, 0.05, 0.04, 0.05, 0.05, 0.05, 0.05, 0.04, 0.05, 0.04, 0.05, 0.04, 0.1, 0.07]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.57
2,28,2,17,17,17,"[0.05, 0.05, 0.05, 0.04, 0.05, 0.06, 0.05, 0.04, 0.05, 0.05, 0.05, 0.05, 0.04, 0.05, 0.04, 0.05, 0.04, 0.1, 0.07]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.57
3,28,3,17,17,17,"[0.05, 0.05, 0.05, 0.04, 0.05, 0.06, 0.05, 0.04, 0.05, 0.05, 0.05, 0.05, 0.04, 0.05, 0.04, 0.05, 0.04, 0.09, 0.07]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.58
4,28,4,17,0,0,"[0.1, 0.06, 0.06, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.04, 0.05, 0.04, 0.05, 0.05, 0.05]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",False,0.800000,0.60
5,28,5,17,0,0,"[0.11, 0.06, 0.06, 0.04, 0.05, 0.05, 0.04, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.04, 0.05, 0.05, 0.05]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",False,0.666667,0.60
6,28,6,17,0,0,"[0.11, 0.06, 0.06, 0.04, 0.05, 0.05, 0.04, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.05, 0.05, 0.05]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",False,0.571429,0.60
7,28,7,17,0,0,"[0.11, 0.06, 0.05, 0.04, 0.05, 0.05, 0.04, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.05, 0.05, 0.05]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",False,0.500000,0.60
8,28,8,17,0,1,"[0.09, 0.07, 0.06, 0.05, 0.06, 0.06, 0.04, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.04, 0.05, 0.05, 0.05, 0.05]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",False,0.444444,0.60
9,28,9,17,1,1,"[0.08, 0.09, 0.05, 0.06, 0.04, 0.06, 0.05, 0.05, 0.04, 0.05, 0.05, 0.06, 0.04, 0.05, 0.04, 0.05, 0.05, 0.05, 0.05]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",False,0.400000,0.60


Total SKTR move cost for case 28: 43.0000
